# Random Graphs
**Erdos-Renyi random graph**. This graph is composed by $N$ nodes and each connection between pairs of nodes are stablished according to probability $p$. This is a typical case of a Bernoulli process.

Notice that the average degree of the generated network is not exactly as the input average degree. The input degree is the expected number of connections.

`nx.gnp_random_graph`: Returns a $G_{n,k}$ random graph, also known as an Erdős-Rényi random graph or a binomial graph.


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from scipy.stats import poisson
from scipy.stats import binom
import sdeint

In [ ]:
N = 100
av_degree = 8
p = av_degree/(N-1)
G = nx.gnp_random_graph(N, p, seed=None, directed=False)

In [ ]:
N = len(G)
M = G.number_of_edges()
print('Number of nodes:', N)
print('Number of edges:', M)
print('Average degree:', 2*M/N)

In [ ]:
pos = nx.fruchterman_reingold_layout(G);
plt.figure(figsize=(8,8));
plt.axis("off");
nx.draw_networkx_nodes(G, pos, node_size=300, node_color="black");
nx.draw_networkx_edges(G, pos, alpha=0.500);
nx.draw_networkx_labels(G, pos, font_color="white");
plt.show();

In [ ]:
def degree_distribution(GER):
    vk = dict(GER.degree())
    vk = list(vk.values()) # we get only the degree values
    maxk = np.max(vk)
    mink = np.min(min)
    kvalues= np.arange(0,maxk+1) # possible values of k
    Pk = np.zeros(maxk+1) # P(k)
    for k in vk:
        Pk[k] = Pk[k] + 1
    Pk = Pk/sum(Pk) # the sum of the elements of P(k) must to be equal to one
    return kvalues,Pk

In [ ]:
ks, Pk = degree_distribution(G)

In [ ]:
plt.figure()
plt.plot(ks,Pk,'bo', label='Data')
plt.xlabel("k", fontsize=20)
plt.ylabel("P(k)", fontsize=20)
plt.title("Degree distribution", fontsize=20)
plt.grid(True)
plt.savefig('degree_dist.eps') #save the figure into a file

## Fitting of. binomial distribution
pk = binom.pmf(ks, N-1, p)
plt.plot(ks, pk, 'r', label='Binomial distribution')
plt.legend()
plt.show(True)

### Poisson Distribution

For Poisson Distribution (previously demonstrated with binomial dist) use `scipy.stats.poisson.pmf(x, μ)`, where the probability mass function for poisson is: $$\exp(-\mu)\frac{\mu^k}{k!}$$

In [ ]:
#example
z = 0.6
N = 1000
x = range(5)
plt.plot(x, poisson.pmf(x, z))

## Expected size of the largest two components: plot (z, expected size)

Recall from `Support_class_6.ipynb` that 

`sorted([G.subgraph(c) for c in nx.connected_components(G)], key = len, reverse=True)`, 

returns nodes contained in each component listed from largest to smallest

In [ ]:
z = np.arange(0.1, 3, 0.1) #Expected average degree
N=100
realisations = 20
p = z/N
S1 = np.zeros((len(p), realisations))
S2 = np.zeros((len(p), realisations))

for i in range(realisations):
    for index, j in enumerate(p):
        G=nx.gnp_random_graph(N,j) # build a graph
        Components = sorted([G.subgraph(c) for c in nx.connected_components(G)], key = len, reverse=True)
        S1[index,i] = len(Components[0]) #size of largest connected component 
        if len(Components) == 1: #check if graph is fully connected
            S2[index,i] = 0
        else:
            S2[index,i] = len(Components[1]) #size of 2nd largest component

In [ ]:
Expected_S1 = np.mean(S1, axis = 1)
Expected_S2 = np.mean(S2, axis = 1)

In [ ]:
plt.figure(figsize = (10,5))
plt.plot(z, Expected_S1, label = "largest component size")
plt.plot(z, Expected_S2, label = "second largest component size")
plt.ylabel("Expected Size", fontsize = 20)
plt.xlabel("z", fontsize = 20)
plt.title("Expected size of the largest two components for N =100", fontsize = 20)
plt.legend(loc = "upper left", fontsize = 16)

## Expected local clustering coefficient
The clustering coefficient of a random network is equal to $p$.

In [ ]:
avc = nx.average_clustering(G)
print("Average clustering:", avc, 'and, p = ', j)

In [ ]:
z = np.arange(0.1, 10, 0.1)
N=100
realisations = 20
p = z/N
CC = np.zeros((len(p), realisations))
for i in range(realisations):
    for index, j in enumerate(p):
        G=nx.gnp_random_graph(N,j) # build a graph
        c=nx.average_clustering(G)
        CC[index,i] = c

In [ ]:
average_CC = np.mean(CC, axis = 1)

In [ ]:
plt.figure(figsize = (10,5))
plt.plot(z, average_CC, "o")
plt.ylabel("Expected clustering coef", fontsize = 20)
plt.xlabel("z", fontsize = 20)

## Wigner semi-circle law
Spectrum of adjacency matrix $A$: set of eigenvalues of a matrix

Plot eigenvalue against spectral density (and compare this to wigner semi-circle law).

Use `nx.to_numpy_matrix(G)` to get the adjacency matrix (in numpy format). There exists `nx.adjacency_matrix(G)` which creates a **sparse matrix** -- better for large data sets

In [ ]:
z = 10
N = 100
p = z/N
G = nx.gnp_random_graph(N,p)

In [ ]:
#nx.draw(G)

In [ ]:
A = nx.to_numpy_matrix(G)
evals, evecs = np.linalg.eig(A/np.sqrt(N))

In [ ]:
spectral_density = stats.gaussian_kde(evals, bw_method = 0.05)
λ = np.linspace(evals.min(), evals.max(), N)

In [ ]:
plt.plot(λ, spectral_density(λ))
plt.xlabel('λ', fontsize = 20)
plt.ylabel('ρ(λ)', fontsize = 20)

### Wigner semi cirle
A **wigner matrix** is a real, symmetric matrix with iid entries $a_{ij}$, $\mathbb{E}(a_{ij}) =0$, var$(a_{ij}) = \sigma^2$.

**Wigner semi-circle law** - the spectral density $\rho_N$ of the matrix $\frac{A}{\sqrt{N}}$ converges with probability 1 to: $$ (2 \pi \sigma^2)^{-1} \sqrt{4 \sigma^2 - \lambda^2} \ \text{,if} \ |\lambda|<2\sigma$$

In [ ]:
def semicircle(σ, μ, λ):
    wigner = np.zeros(len(λ))
    for j in range(len(λ)):
        if abs(λ[j])< 2*σ:
            wigner[j] = (1/(2*np.pi*σ**2))*np.sqrt(4*σ**2 - λ[j]**2)
        else:
            wigner[j] = 0
    return wigner

In [ ]:
ee = np.linspace(-0.7, 0.7, 100000)

In [ ]:
AA = A.flatten()
σ = np.std(AA)
μ = np.mean(AA)
wigner = semicircle(σ, μ, ee)

In [ ]:
plt.figure(figsize = (10,5))
plt.plot(λ, spectral_density(λ))
plt.plot(ee, wigner)
plt.xlabel('λ', fontsize = 20)
plt.ylabel('ρ(λ)', fontsize = 20)
plt.xlim([-0.75, 0.75])